In [1]:
import unittest
import os
os.environ["MKL_NUM_THREADS"] = "4" # export MKL_NUM_THREADS=6
from regcoil import *
from shape_gradient_dask import *
import numpy as np
import logging
%load_ext line_profiler
import dask.array as da
np.show_config()
os.chdir('../')
os.getcwd()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/rrobin/anaconda3/envs/fusion/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/rrobin/anaconda3/envs/fusion/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/rrobin/anaconda3/envs/fusion/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/rrobin/anaconda3/envs/fusion/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/rrobin/anaconda3/envs/fusion/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/rrobin/anaconda3/envs/fusion/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/rrobin/anaconda3/envs/fusion/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/rrobin/anaconda3/envs/fusion/include']


'/home/rrobin/Documents/shape_opti/code'

In [2]:
from dask.distributed import Client
client = Client(processes=False,
                n_workers=2, threads_per_worker=3)
client.scheduler_info()['services']

{'dashboard': 8787}

In [3]:
print(os.getcwd())
shape_grad=Shape_gradient_dask('config.ini')
def f(param_array):
    (m,n,Rmn,Zmn)=shape_grad.S_parametrization
    R=param_array[:len(m)]
    Z=param_array[len(m):]
    param=((m,n,R,Z))
    return shape_grad.compute_cost(param)['cost']
def gradf(param_array):
    (m,n,Rmn,Zmn)=shape_grad.S_parametrization
    R=param_array[:len(m)]
    Z=param_array[len(m):]
    param=((m,n,R,Z))
    return shape_grad.compute_gradient(param)['shape_gradient']

/home/rrobin/Documents/shape_opti/code


In [4]:
%lprun -u 1 -f shape_grad.compute_cost shape_grad.compute_cost(shape_grad.S_parametrization)

Timer unit: 1 s

Total time: 4.99988 s
File: /home/rrobin/Documents/shape_opti/code/main/shape_gradient_dask.py
Function: compute_cost at line 143

Line #      Hits         Time  Per Hit   % Time  Line Contents
   143                                               def compute_cost(self,paramS):
   144         1          0.0      0.0      0.0          result={}
   145         1          0.2      0.2      4.9          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
   146         1          0.2      0.2      4.3          T=vector_field.get_tensor_distance(S,self.Sp,self.rot_tensor)
   147                                                   #LS=vector_field.compute_LS(T,self.matrixd_phi,S.dpsi,self.rot_tensor,self.Sp.n)
   148         1          0.0      0.0      0.5          Qj=vector_field.compute_Qj(self.matrixd_phi,S.dpsi,S.dS)
   149                                                  
   150         1          0.0      0.0      0.0          dask_dpsi=da.from_array(S.

In [5]:
%lprun -u 1 -f shape_grad.compute_gradient shape_grad.compute_gradient(shape_grad.S_parametrization)

Timer unit: 1 s

Total time: 67.7442 s
File: /users/home/robin/stellacageShapeOpti/code/main/shape_gradient_dask.py
Function: compute_gradient at line 57

Line #      Hits         Time  Per Hit   % Time  Line Contents
    57                                               def compute_gradient(self,paramS):
    58         1          0.0      0.0      0.0          result={}
    59         1          0.2      0.2      0.4          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    60         1          2.1      2.1      3.0          theta,dtildetheta,dtheta,dSdtheta=S.get_theta_pertubation()
    61         1          0.2      0.2      0.3          T=vector_field.get_tensor_distance(S,self.Sp,self.rot_tensor)
    62                                                   #LS=vector_field.compute_LS(T,self.matrixd_phi,S.dpsi,self.rot_tensor,self.Sp.n)
    63         1          0.0      0.0      0.0          Qj=vector_field.compute_Qj(self.matrixd_phi,S.dpsi,S.dS)
    64      

In [6]:
eps=1e-6
S_param_complete=shape_grad.S_parametrization
ls=len(S_param_complete[0])
        
#we apply one pertubation
perturb=(2*np.random.random(2*ls)-1)
S_param=np.concatenate((S_param_complete[2],S_param_complete[3]))
new_param=S_param+eps*perturb
t0=time.time()
old_cost=f(S_param)
new_cost=f(new_param)
gradient=gradf(S_param)
print('time used : {}s'.format(time.time()-t0))

time used : 71.58100700378418s


In [7]:
print('numerical cost obtained: {:10e} and {:10e}'.format(old_cost,new_cost))
print('numerical gradient : {:10e}'.format((new_cost-old_cost)/eps))
print('gradient computed : {:10e}'.format(np.dot(gradient,perturb)))

numerical cost obtained: 1.232865e+01 and 1.232864e+01
numerical gradient : -1.000059e+01
gradient computed : -1.000151e+01


In [ ]:
import scipy.optimize
optimize_shape=scipy.optimize.minimize(f, S_param, method='CG', jac=gradf,options={'maxiter':100,'return_all':True})

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fcdf968f520>>, <Task finished name='Task-162364' coro=<Worker.heartbeat() done, defined at /users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/distributed/worker.py:941> exception=RuntimeError('dictionary changed size during iteration')>)
Traceback (most recent call last):
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/tornado/ioloop.py", line 765, in _discard_future_result
    future.result()
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/distributed/worker.py", line 955, in heartbeat
    executing={
  File "/users/home/robin/miniconda3/envs/shape_opti/lib/python3.9/site-packages/distributed/

In [ ]:
print(optimize_shape)